# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapa'a,22.0752,-159.3190,82.38,82.0,100.0,17.27,US,1.717447e+09
1,1,kodiak,57.7900,-152.4072,45.91,87.0,100.0,6.91,US,1.717448e+09
2,2,san patricio,28.0170,-97.5169,91.35,60.0,100.0,21.34,US,1.717448e+09
3,4,thompson,55.7435,-97.8558,59.16,55.0,75.0,8.05,CA,1.717448e+09
4,5,bredasdorp,-34.5322,20.0403,58.21,77.0,100.0,10.51,ZA,1.717448e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    global_extent = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    color = "City",
    s = "Humidity"
)

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[
    (city_data_df["Max Temp"] < 80) &
    (city_data_df["Max Temp"] > 70) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] < 30) &
    (city_data_df["Humidity"] < 80) &
    (city_data_df["Humidity"] > 40)
].reset_index(drop=True)

ideal_city_df = ideal_city_df.dropna(how="all").reset_index(drop=True)
display(ideal_city_df)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,122,mersa matruh,31.3525,27.2453,75.40,73.0,0.0,6.91,EG,1.717448e+09
1,286,vrontados,38.4089,26.1328,75.06,73.0,0.0,1.14,GR,1.717448e+09
2,300,gargalianoi,37.0667,21.6333,74.35,67.0,0.0,3.49,GR,1.717448e+09
3,324,kaliua,-5.0606,31.7936,70.99,53.0,3.0,4.68,TZ,1.717448e+09
4,385,mahajanga,-15.7167,46.3167,73.42,68.0,0.0,2.30,MG,1.717448e+09
5,401,nishinoomote,30.7333,131.0000,71.46,64.0,29.0,7.36,JP,1.717448e+09
6,406,morondava,-20.2833,44.2833,74.32,71.0,1.0,2.77,MG,1.717448e+09
7,417,jamestown,42.0970,-79.2353,77.45,64.0,20.0,8.05,US,1.717448e+09
8,433,kinkala,-4.3614,14.7644,73.60,79.0,5.0,2.77,CG,1.717448e+09
9,557,lenino,45.2986,35.7771,70.38,71.0,1.0,7.61,UA,1.717448e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mersa matruh,EG,31.3525,27.2453,73.0,
1,vrontados,GR,38.4089,26.1328,73.0,
2,gargalianoi,GR,37.0667,21.6333,67.0,
3,kaliua,TZ,-5.0606,31.7936,53.0,
4,mahajanga,MG,-15.7167,46.3167,68.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1,
    "radius": radius,
    "filter": None,
    "bias": None
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row.Lat
    longitude = row.Lng
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" 
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    if name_address.status_code == 200:
        name_address = name_address.json()
    else:
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        pass
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    time.sleep(1)

# Display sample data
hotel_df.head()

Starting hotel search
mersa matruh - nearest hotel: صقور
vrontados - nearest hotel: Aegean Sea
gargalianoi - nearest hotel: Messinian Horizons
kaliua - nearest hotel: No hotel found
mahajanga - nearest hotel: Vahiny Hotel
nishinoomote - nearest hotel: ゼウスハウス
morondava - nearest hotel: Hotel Sharon Menabe
jamestown - nearest hotel: DoubleTree Jamestown
kinkala - nearest hotel: hotel
lenino - nearest hotel: Восток
brejo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mersa matruh,EG,31.3525,27.2453,73.0,صقور
1,vrontados,GR,38.4089,26.1328,73.0,Aegean Sea
2,gargalianoi,GR,37.0667,21.6333,67.0,Messinian Horizons
3,kaliua,TZ,-5.0606,31.7936,53.0,No hotel found
4,mahajanga,MG,-15.7167,46.3167,68.0,Vahiny Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)